In [1]:
import pandas as pd
import os

In [3]:
df = pd.read_csv('/Users/danielepica/Documents/GitHub/recommender_system_llm/src/books_recommendation.csv')
df = df[:100]

In [4]:
df.head()

,books_info
0,title:The Church of Christ: A Biblical Ecclesi...
1,title:No place to cry: The hurt and healing of...
2,"title:Night World: Daughters Of Darkness, desc..."
3,"title:The Rabbi's Cat, description:Gaining the..."
4,"title:From Potter's Field, description:The six..."


In [5]:
#!pip install chromadb langchain
from langchain.embeddings import OllamaEmbeddings
from langchain.llms import Ollama
from langchain.vectorstores import Chroma
import os

embeddings = OllamaEmbeddings(model='mxbai-embed-large', show_progress=True)
llm = Ollama(model='llama3')

#create a new Chroma vector store (index)
persist_directory = 'db_persist'
#Check if the directory already exists
if os.path.exists(persist_directory):
    #Remove the directory and its contents
    import shutil
    shutil.rmtree(persist_directory)

recommend_db = Chroma(embedding_function=embeddings, persist_directory=persist_directory)

# Crea embeddings riga per riga e salva in Chroma
for idx, row in df.iterrows():
    # Estrarre l'informazione combinata
    document_text = row['books_info']
   
    # Aggiungere il documento alla base di dati vettoriale Chroma
    recommend_db.add_texts(texts=[document_text], metadatas=[{'user_id': idx}])

# Persistenza dei dati
recommend_db.persist() #NON è PIù NECESSARIO


OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 20.36it/s]
/opt/anaconda3/envs/LangChain/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [8]:
#1 Metodo
query = "If i liked Economics in one lesson, What other books like it could you suggest to me?"
docs = recommend_db.similarity_search(query, k=5)
docs

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 11.53it/s]


[Document(metadata={'user_id': 5}, page_content="title:Economics in one lesson, description:With over a million copies sold, Economics in One Lesson is an essential guide to the basics of economic theory. A fundamental influence on modern libertarianism, Hazlitt defends capitalism and the free market from economic myths that persist to this day. Considered among the leading economic thinkers of the “Austrian School,” which includes Carl Menger, Ludwig von Mises, Friedrich (F.A.) Hayek, and others, Henry Hazlitt (1894-1993), was a libertarian philosopher, an economist, and a journalist. He was the founding vice-president of the Foundation for Economic Education and an early editor of The Freeman magazine, an influential libertarian publication. Hazlitt wrote Economics in One Lesson, his seminal work, in 1946. Concise and instructive, it is also deceptively prescient and far-reaching in its efforts to dissemble economic fallacies that are so prevalent they have almost become a new orthod

In [ ]:
from langchain.chains.retrieval_qa.base import RetrievalQA

#2 Metodo
qa = RetrievalQA.from_chain_type(llm=llm, chain_type = "stuff", retriever = recommend_db.as_retriever(), return_source_documents=True, verbose=True)

In [ ]:
query = "I'm looking for an drama book. What could you suggest to me?"
result = qa({"query": query})
result['result']

In [6]:
type(recommend_db)

langchain_community.vectorstores.chroma.Chroma